# Assignment_3 - Missing data in Titanic - Jorge Alarcon Sanz

#### 1. What is the meaning of the missing data for these two columns (Age and Cabin). Did the creators of the dataset leave missing values on purpose or not?

#### 2. If you think there is any column whose missing data was not left there on purpose and that we could benefit from having, think of some strategy to impute it (https://www.wikiwand.com/en/Imputation_(statistics)). The most trivial strategy is to impute the missing values with some global aggregation: the mean, the median, or the mode. 
#### Can you do better?

#### <u>The Age column<u/>:

1.We can see that the 19,9% of the total observations in the field "Age" are NAs. I think the NAs related to this field are real missing values since a lot of personal details related with the people who traveled in the Titanic were lost. Also, some of those personal details are really difficult to investigate and find a real data about it. However, there is a list in wikipedia (https://en.wikipedia.org/wiki/Passengers_of_the_RMS_Titanic#Passenger_list), where you can find the ages for almost all the passengers. In spite of this, the most recognized and best valued web page about acurate data about Titanic (http://www.encyclopedia-titanica.org/titanic-passenger-list) contains a total of 32.9% of the data on the age of the passengers and staff as missed values.
By analyzing the dataset itself, there is not correlation between missing values and the other features what means that in case that the creators of the dataset leave missing values on purpose they did it randomly. 

Finally, we have to take into consideration that the events occurred in 1912 (26 years before the first computer); at that time all records about passengers were taken manually and it could happen many situations to lose a record: 1º. The passenger did not have a birth certificate or even they did not know their age, 2º. That the age in the registry in the purchase of the ticket had not been facilitated by purpose on origin by confidenciality and they did not have a way to verify his age after his death (around 120 over the 177 NAs are passengers who died and match with this hypothesis).

For all these reasons, **I consider that the lack of the 20% of the "Age" values in the database is justified and is not due to the manipulation of the creators**. 


2.Being a variable that seems relevant for the prediction of the death of the passage as well as having enough sample for the prediction of the NAs of the variable "Age", **I decided to impute the NAs through a linear regression**.

The dataset has 714 complete observations in this field and I consider it is fisible develop a regression model in order to predict the values for the 177 NAs. I've decided to develop a linear regression for simplicity reasons. First I will create a train set with all not NA observations in age. Then I will split this new df in a train and validation datasets (80%-20%). After this, I will use the MSE as main error function in the validation sample and I will compute the R2.  

#### <u>The Cabin column<u/>:

1.We can see that the 77.1% of the total observations in the field "Cabin" are NAs. In relation to this field there are many considerations to take into account. 
First of all, I was able to discover according to the "Encyclopedia titanica" that the only reliable and authorized data concerning the cabin is the one referring to first class passengers (https://www.encyclopedia-titanica.org/cabins.html). The passengers of first class represent the 24.6% and the percentage of non-NAs in the cabin field is around 22.9%. That means that the Kaggle database includes less data that the reliable and authorized one. This is the first reason why I consider that the creators of the database are manipulating the NAs on purpose.
Another information that make me suspect about the manipulation of missing values by the creators of the database is that, by data I have collected from different sources, there is a statistical significance between the variable Survival and Pclass, that could means that the location of all the cabins in the first class could increase the efficiency of the model easily and maybe the creators of the database would like to block the development of a predictive model that included the use of this variable to increase the difficulty of the challenge.

For these reasons, **I consider that the lack of the 77% of the "Cabin" values in the database is not justified and is due to the manipulation of the creators**. 

2.In my personal and unexperienced opinion, the way to treat with the NAs of the variable "Cabin" would be by spliting the variable in two new variabls that will be: (1st) the letter and (2nd) the number of it.
I would develop a random forest to predict the letter and a xgboots to predict the numbers of the cabins. On one hand, these high accuracy models will help to avoid as much as possible the errors due to the the small sample. On the other hand, the prediction of these models will be very skewed by the lack of sample.
Due to:

    A. The difficulty in carrying out a good "Cabin" variable with great efficiency in the prediction of NA and, 
    B. The uncertainty as a significant variable for the regression of a predictive model that detects the survival
**I consider that the best option is not to use the variable as an explanatory variable for a predictive model based on this dataset**.

# <span style="color:red">Let's start coding!!</span>

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [2]:
import pandas as pd
import os
import statsmodels.api as sm
import xlsxwriter


C:\Users\soyja\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


We read the titanic csv and we will save it as df

In [3]:
# Dataset from https://www.kaggle.com/c/titanic/download/train.csv
df = pd.read_csv(
    "Titanic_Age_train_v2.csv", sep=',', index_col="PassengerId",
    #decimal=','
)
df.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


I will analyse the number of NAs by columns...

In [4]:
df.notnull().sum()

Survived    891
Pclass      891
Name        891
Sex         891
Age         714
SibSp       891
Parch       891
Ticket      891
Fare        891
Cabin       204
Embarked    889
dtype: int64

...and the number of complete cases

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

### Feature engeniering

Let's start understanding the correlations of our original variables

In [6]:
df.corr()

,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


We are going to use the title name so the first step is to extract it from the name (Mr, Mrs...)

In [7]:
df[['Name','Name_type']] = df['Name'].str.split(',', expand=True)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_type
PassengerId,,,,,,,,,,,,
1,0,3,Braund,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr. Owen Harris
2,1,1,Cumings,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs. John Bradley (Florence Briggs Thayer)
3,1,3,Heikkinen,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss. Laina
4,1,1,Futrelle,female,35.0,1,0,113803,53.1000,C123,S,Mrs. Jacques Heath (Lily May Peel)
5,0,3,Allen,male,35.0,0,0,373450,8.0500,NaN,S,Mr. William Henry


In [8]:
df[['Name_type','Name_1']] = df['Name_type'].str.split('.', expand=True).iloc[:, :2]
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_type,Name_1
PassengerId,,,,,,,,,,,,,
1,0,3,Braund,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,Owen Harris
2,1,1,Cumings,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,John Bradley (Florence Briggs Thayer)
3,1,3,Heikkinen,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,Laina
4,1,1,Futrelle,female,35.0,1,0,113803,53.1000,C123,S,Mrs,Jacques Heath (Lily May Peel)
5,0,3,Allen,male,35.0,0,0,373450,8.0500,NaN,S,Mr,William Henry


We assign a numerical code per each type of title

In [9]:
np.unique(df.Name_type)

array([' Capt', ' Col', ' Don', ' Dr', ' Jonkheer', ' Lady', ' Major',
       ' Master', ' Miss', ' Mlle', ' Mme', ' Mr', ' Mrs', ' Ms', ' Rev',
       ' Sir', ' the Countess'], dtype=object)

I tried to input for each class a numerical value through the following formula, the problem was that since the numbers do not follow a rational distribution the model does not improve. Because of this, I decided to assign manually the value of each Name_type. You can see the code below(**).

In [10]:
#df.Name_type = pd.Categorical(df.Name_type)
#df['Name_code'] = df.Name_type.cat.codes

At this point we already have the column Name_type with the type of pre-name and his numerical code

Now, first we are going to subset the NAs in the Age column and we are going to create a new data set (Age test.set)

In [11]:
Titanic_Age_NA = df.loc[pd.isnull(df.Age), :]
Titanic_Age_NA.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_type,Name_1
PassengerId,,,,,,,,,,,,,
6,0,3,Moran,male,NaN,0,0,330877,8.4583,NaN,Q,Mr,James
18,1,2,Williams,male,NaN,0,0,244373,13.0000,NaN,S,Mr,Charles Eugene
20,1,3,Masselmani,female,NaN,0,0,2649,7.2250,NaN,C,Mrs,Fatima
27,0,3,Emir,male,NaN,0,0,2631,7.2250,NaN,C,Mr,Farred Chehab
29,1,3,O'Dwyer,female,NaN,0,0,330959,7.8792,NaN,Q,Miss,"Ellen ""Nellie"""


Second, we subset by no-NAs in Age column and we create a data set (Age train.set)

In [12]:
Titanic_Age_train = df.loc[pd.notnull(df.Age), :]
Titanic_Age_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_type,Name_1
PassengerId,,,,,,,,,,,,,
1,0,3,Braund,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,Owen Harris
2,1,1,Cumings,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,John Bradley (Florence Briggs Thayer)
3,1,3,Heikkinen,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,Laina
4,1,1,Futrelle,female,35.0,1,0,113803,53.1000,C123,S,Mrs,Jacques Heath (Lily May Peel)
5,0,3,Allen,male,35.0,0,0,373450,8.0500,NaN,S,Mr,William Henry


In [13]:
Titanic_Age_train.to_csv('Titanic_Age_train_v2.csv')

In [14]:
Titanic_Age_NA.to_csv('Titanic_Age_test_v2.csv')

### Other data transformations

In [15]:
df_prepared = pd.read_csv(
    "Titanic_Age_train_v2.csv", sep=';', index_col = "PassengerId"
)

In [16]:
df_NA_prepared = pd.read_csv(
    "Titanic_Age_test_v2.csv", sep=';', index_col = "PassengerId"
)

In [17]:
df_prepared.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_type,Name_code,Name_1
PassengerId,,,,,,,,,,,,,,
1,0,3,Braund,male,22.0,1,0,A/5 21171,7.25,NaN,S,Mr,6,Owen Harris
2,1,1,Cumings,female,38.0,1,0,PC 17599,712.833,C85,C,Mrs,8,John Bradley (Florence Briggs Thayer)
3,1,3,Heikkinen,female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S,Miss,2,Laina
4,1,1,Futrelle,female,35.0,1,0,113803,53.1,C123,S,Mrs,8,Jacques Heath (Lily May Peel)
5,0,3,Allen,male,35.0,0,0,373450,8.05,NaN,S,Mr,6,William Henry


Let's plot the correlation again with the new variable:

In [18]:
df_prepared.corr()

,Survived,Pclass,Age,SibSp,Parch,Name_code
Survived,1.000000,-0.359653,-0.077221,-0.017358,0.093317,-0.164471
Pclass,-0.359653,1.000000,-0.369226,0.067247,0.025683,-0.125459
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.507205
SibSp,-0.017358,0.067247,-0.308247,1.000000,0.383820,-0.199259
Parch,0.093317,0.025683,-0.189119,0.383820,1.000000,-0.130670
Name_code,-0.164471,-0.125459,0.507205,-0.199259,-0.130670,1.000000


Through this correlation we can see that the transformations 

Creamos un nuevo df con las variables numéricas:

In [19]:
Titanic_Age_train_numeric = df_prepared._get_numeric_data()

In [20]:
Titanic_Age_train_numeric.head()

,Survived,Pclass,Age,SibSp,Parch,Name_code
PassengerId,,,,,,
1,0,3,22.0,1,0,6
2,1,1,38.0,1,0,8
3,1,3,26.0,0,0,2
4,1,1,35.0,1,0,8
5,0,3,35.0,0,0,6


Now we going to split the train data set into 80/20 in order to create a prediction/validation datasets

In [21]:
Titanic_Age_train_numeric_C = Titanic_Age_train_numeric.loc[:, Titanic_Age_train_numeric.columns != 'Age']

Split the data into training/testing sets

In [22]:
len(Titanic_Age_train_numeric_C)*0.2

142.8

In [23]:
Titanic_Age_train_set = Titanic_Age_train_numeric_C[:-143]
Titanic_Age_test_set = Titanic_Age_train_numeric_C[-143:]

In [24]:
Titanic_Age_train_set.head()

,Survived,Pclass,SibSp,Parch,Name_code
PassengerId,,,,,
1,0,3,1,0,6
2,1,1,1,0,8
3,1,3,0,0,2
4,1,1,1,0,8
5,0,3,0,0,6


Split the targets into training/testing sets

In [25]:
Titanic_Age_train_target = Titanic_Age_train_numeric.Age[:-143]
Titanic_Age_test_target = Titanic_Age_train_numeric.Age[-143:]

Importamos los datos de sklearn and we create linear regression object

In [26]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
regr = linear_model.LinearRegression()

Train the model using the training sets

In [27]:
model = regr.fit(Titanic_Age_train_set, Titanic_Age_train_target)

We make predictions using the testing set

In [28]:
Titanic_Age_pred = regr.predict(Titanic_Age_test_set)

We print the coefficients

In [29]:
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [-4.47774975 -6.39551524 -3.19461274 -0.93267132  2.15923942]


We compute the mean squared error

In [31]:
print("Mean squared error: %.2f"
      % mean_squared_error(Titanic_Age_test_target, Titanic_Age_pred))

Mean squared error: 118.24


Here I will run the R2

In [32]:
print('Variance score: %.2f' % r2_score(Titanic_Age_test_target, Titanic_Age_pred))

Variance score: 0.41


#### Now it's time to predict the NAs!

In [33]:
df_NA_prepared.Age=round(regr.intercept_ + 
                         df_NA_prepared.Survived*regr.coef_[0] + 
                         df_NA_prepared.Pclass*regr.coef_[1] + 
                         df_NA_prepared.SibSp*regr.coef_[2] + 
                         df_NA_prepared.Parch*regr.coef_[3] + 
                         df_NA_prepared.Name_code*regr.coef_[4]) 

In [34]:
df_NA_prepared.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_type,Name_code,Name_1
PassengerId,,,,,,,,,,,,,,
6,0,3,Moran,male,30.0,0,0,330877,84.583,NaN,Q,Mr,6,James
18,1,2,Williams,male,32.0,0,0,244373,13.0,NaN,S,Mr,6,Charles Eugene
20,1,3,Masselmani,female,30.0,0,0,2649,7.225,NaN,C,Mrs,8,Fatima
27,0,3,Emir,male,30.0,0,0,2631,7.225,NaN,C,Mr,6,Farred Chehab
29,1,3,O'Dwyer,female,17.0,0,0,330959,78.792,NaN,Q,Miss,2,"Ellen ""Nellie"""


In [35]:
df_complete_age = df_prepared.append(df_NA_prepared).sort_index()

In [39]:
df_complete_age.head(6)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_type,Name_code,Name_1
PassengerId,,,,,,,,,,,,,,
1,0,3,Braund,male,22.0,1,0,A/5 21171,7.25,NaN,S,Mr,6,Owen Harris
2,1,1,Cumings,female,38.0,1,0,PC 17599,712.833,C85,C,Mrs,8,John Bradley (Florence Briggs Thayer)
3,1,3,Heikkinen,female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S,Miss,2,Laina
4,1,1,Futrelle,female,35.0,1,0,113803,53.1,C123,S,Mrs,8,Jacques Heath (Lily May Peel)
5,0,3,Allen,male,35.0,0,0,373450,8.05,NaN,S,Mr,6,William Henry
6,0,3,Moran,male,30.0,0,0,330877,84.583,NaN,Q,Mr,6,James
